# Common Architectures in Keras    Shallow logistic regression, custom RBF layer, RBM, CNN, feature hierarchy, and pretrained feature extractor using `tf.keras`.

In [ ]:
    # !pip install tensorflow scikit-learn matplotlib    import tensorflow as tf    from tensorflow import keras    from tensorflow.keras import layers    import numpy as np    import matplotlib.pyplot as plt    from sklearn.datasets import load_digits    from sklearn.model_selection import train_test_split    from sklearn.preprocessing import MinMaxScaler    tf.keras.utils.set_random_seed(123)    

## Shallow Model (logistic regression)

In [ ]:
    X = np.random.randn(400, 2)    true_w = np.array([[1.5], [-0.7]])    logits = X @ true_w + 0.2    y = (1 / (1 + np.exp(-logits)) > 0.5).astype(np.int32).ravel()    shallow = keras.Sequential([layers.Input(shape=(2,)), layers.Dense(1, activation='sigmoid')])    shallow.compile(optimizer=keras.optimizers.SGD(learning_rate=0.5), loss='binary_crossentropy', metrics=['accuracy'])    shallow.fit(X, y, epochs=60, verbose=0)    acc = shallow.evaluate(X, y, verbose=0)[1]    print(f"Logistic regression accuracy: {acc:.3f}")    

## RBF-style hidden layer (trainable centers)

In [ ]:
    class RBFLayer(layers.Layer):        def __init__(self, units, sigma=1.0, **kwargs):            super().__init__(**kwargs)            self.units = units            self.sigma = sigma        def build(self, input_shape):            self.centers = self.add_weight(                name='centers',                shape=(self.units, int(input_shape[-1])),                initializer='random_normal',                trainable=True            )        def call(self, inputs):            diff = tf.expand_dims(inputs, 1) - tf.expand_dims(self.centers, 0)            dist_sq = tf.reduce_sum(tf.square(diff), axis=2)            return tf.exp(-dist_sq / (2.0 * self.sigma ** 2))    inputs = keras.Input(shape=(2,))    rbf_feat = RBFLayer(8, sigma=1.5)(inputs)    rbf_out = layers.Dense(1, activation='sigmoid')(rbf_feat)    rbf_model = keras.Model(inputs, rbf_out)    rbf_model.compile(optimizer=keras.optimizers.Adam(0.05), loss='binary_crossentropy', metrics=['accuracy'])    rbf_model.fit(X, y, epochs=120, verbose=0)    rbf_acc = rbf_model.evaluate(X, y, verbose=0)[1]    print(f"RBF-like model accuracy: {rbf_acc:.3f}")    

## Restricted Boltzmann Machine (RBM)    Minimal Contrastive Divergence (CD-1) demo on digits data.

In [ ]:
    digits = load_digits()    data = MinMaxScaler().fit_transform(digits.data).astype('float32')    X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)    class SimpleRBM:        def __init__(self, n_visible, n_hidden):            self.W = tf.Variable(tf.random.normal([n_visible, n_hidden], stddev=0.01))            self.vb = tf.Variable(tf.zeros([n_visible]))            self.hb = tf.Variable(tf.zeros([n_hidden]))        def hidden_prob(self, v):            return tf.nn.sigmoid(tf.matmul(v, self.W) + self.hb)        def visible_prob(self, h):            return tf.nn.sigmoid(tf.matmul(h, tf.transpose(self.W)) + self.vb)        def sample_prob(self, probs):            return tf.cast(tf.random.uniform(tf.shape(probs)) < probs, tf.float32)    def train_rbm(rbm, data, epochs=6, batch_size=32, lr=0.05):        ds = tf.data.Dataset.from_tensor_slices(data).shuffle(1024).batch(batch_size)        for _ in range(epochs):            for v0 in ds:                h0_prob = rbm.hidden_prob(v0)                h0 = rbm.sample_prob(h0_prob)                v1_prob = rbm.visible_prob(h0)                h1_prob = rbm.hidden_prob(v1_prob)                dW = tf.matmul(v0, h0_prob, transpose_a=True) - tf.matmul(v1_prob, h1_prob, transpose_a=True)                dvb = tf.reduce_mean(v0 - v1_prob, axis=0)                dhb = tf.reduce_mean(h0_prob - h1_prob, axis=0)                rbm.W.assign_add(lr * dW / tf.cast(tf.shape(v0)[0], tf.float32))                rbm.vb.assign_add(lr * dvb)                rbm.hb.assign_add(lr * dhb)    rbm = SimpleRBM(n_visible=X_train.shape[1], n_hidden=64)    train_rbm(rbm, X_train, epochs=6, batch_size=32, lr=0.05)    sample = X_test[:1]    recon = rbm.visible_prob(rbm.sample_prob(rbm.hidden_prob(sample)))    recon_err = tf.reduce_mean(tf.square(sample - recon)).numpy()    print("RBM recon error:", recon_err)    plt.figure(figsize=(4,2))    plt.subplot(1,2,1); plt.imshow(sample.reshape(8,8), cmap='gray'); plt.title('Original'); plt.axis('off')    plt.subplot(1,2,2); plt.imshow(recon.numpy().reshape(8,8), cmap='gray'); plt.title('Recon'); plt.axis('off')    plt.show()    

## CNN on Synthetic Data

In [ ]:
    # Synthetic image-like data (avoid downloads)    x_train = np.random.rand(500, 28, 28, 1).astype('float32')    y_train = np.random.randint(0, 10, size=(500,))    cnn = keras.Sequential([        layers.Input(shape=(28, 28, 1)),        layers.Conv2D(8, 3, padding='same', activation='relu'),        layers.MaxPooling2D(),        layers.Conv2D(16, 3, padding='same', activation='relu'),        layers.MaxPooling2D(),        layers.Flatten(),        layers.Dense(32, activation='relu'),        layers.Dense(10, activation='softmax')    ])    cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])    cnn.fit(x_train, y_train, epochs=2, batch_size=64, verbose=1)    

## Hierarchical Feature Shapes

In [ ]:
    sample = x_train[0:1]    layer_outputs = [layer.output for layer in cnn.layers if isinstance(layer, layers.Conv2D)]    activation_model = keras.Model(inputs=cnn.input, outputs=layer_outputs)    feats = activation_model(sample)    for i, feat in enumerate(feats, start=1):        print(f"Conv layer {i} feature map shape: {feat.shape}")    

## Pretrained Feature Extractor (MobileNetV2)

In [ ]:
    from tensorflow.keras.applications import MobileNetV2    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input    base = MobileNetV2(weights=None, include_top=False, pooling='avg')  # set weights='imagenet' to download    base.trainable = False    dummy = np.random.rand(1, 224, 224, 3).astype('float32')    dummy_pp = preprocess_input(dummy.copy())    features = base(dummy_pp)    print("Feature vector shape:", features.shape)    